In [1]:
import pandas as pd
import nibabel as nb
import os
import shutil

In [2]:
def organise_target_and_volumeid(df_path='/home/abhijit/Jyotirmay/my_thesis/projects/MC_dropout_quicknat/reports/MC_dropout_quicknat_UKB_v2/UKB/10_1573078374.453554_report.csv'):
    df = pd.read_csv(df_path, index_col=0)
    if df['dataset'].values[1] != 2:
        print('Not a UKB dataset report, not processing!')
        return False
    def replace_func(value):
        string = value[-1]
        return f'Dixon_BH_17s_opp_Dixon_BH_17{string}.nii.gz' if string == 's' else f'Dixon_BH_17s_opp_Dixon_BH_17s{string}.nii.gz'
    df['target_scan_file'] = df['volume_id'].apply(replace_func)
    df['volume_id'] = df['volume_id'].apply(lambda x: x[:-1])
    return df

def concat_partial_model_reports(paths):
    df = None
#     ids = ['10_1573078374.453554', '10_1573225388.879571', '10_1573391294.0298784']
#     paths = [f'/home/abhijit/Jyotirmay/my_thesis/projects/MC_dropout_quicknat/reports/MC_dropout_quicknat_UKB_v2/UKB/{i}_report.csv' for i in ids]
    
    for ix, p in enumerate(paths):
        df_ = organise_target_and_volumeid(p)
        df_ = df_.drop(df_.index[0])
        if ix == 0:
            df = df_
        else:
            df = df.append(df_, ignore_index=True)
    return df

In [ ]:
ids = ['10_1573078374.453554', '10_1573225388.879571', '10_1573391294.0298784']
paths = [f'/home/abhijit/Jyotirmay/my_thesis/projects/MC_dropout_quicknat/reports/MC_dropout_quicknat_UKB_v2/UKB/{i}_report.csv' for i in ids]
df = concat_partial_model_reports(paths)  

In [ ]:
df.to_csv('/home/abhijit/Jyotirmay/my_thesis/projects/MC_dropout_quicknat/reports/MC_dropout_quicknat_UKB_v2/UKB/10_0.0_report.csv')

In [ ]:
df = pd.read_csv('/home/abhijit/Jyotirmay/my_thesis/projects/MC_dropout_quicknat/reports/MC_dropout_quicknat_UKB_v2/UKB/10_0.0_report.csv', index_col=0)

In [ ]:
df

In [3]:
with open('/home/abhijit/Jyotirmay/my_thesis/dataset_groups/whole_body_datasets/UKB/test_volumes.txt') as file_handle:
        volumes_to_use = file_handle.read().splitlines()

In [4]:
len(volumes_to_use)

19853

In [5]:
df = pd.read_csv('/home/abhijit/Jyotirmay/my_thesis/projects/MC_dropout_quicknat/reports/MC_dropout_quicknat_UKB_v2/UKB/0_0.0_concat_report_final.csv', index_col=0)

In [9]:
# have 3all 3 as good files for volume: 5900151_20201_2_0


for ix, v in enumerate(volumes_to_use):
    print("\rProcessing {}/{}.".format(ix, len(volumes_to_use)), end="")
    iou_mean = df[df['volume_id']==v]['iou_mean'].values
    i1, i2 = None, None
    try:
        if len(iou_mean) < 1:
            continue
        elif len(iou_mean) ==1:
            continue
        elif len(iou_mean)==2:
            i1, i2 = iou_mean
            if i1>i2:
                idx = i2
            else:
                idx = i1
            df.drop(df.loc[(df['volume_id']==v) & (df['iou_mean']==idx)].index, inplace=True)
        elif len(iou_mean) == 3:
            df.drop(df.loc[df['volume_id']==v].index, inplace=True)
        else:
            print('coming to else part!!!')
            continue
            
                
#         if len(targets)>1:
#             i1, i2 = df[df['volume_id']==v]['iou_mean'].values
#             if i1>i2:
#                 idx = 0
#             else:
#                 idx = 1
#             target = targets[idx]
#         elif len(targets)==0:
#             continue
#         else:
#             target = targets[0]

#     #     input_file = '/home/abhijit/nas_drive/Data_WholeBody/UKBiobank/body/body_nifti/'+str(v)+'/'+target
#     #     gt_file = None
#     #     f_paths.append([input_file, gt_file])
#         postfix = target.split('.')[0][-1]
#         shutil.copy2(f'/home/abhijit/nas_drive/Abhijit/Jyotirmay/ukb_processed/data/{v}{postfix}.nii.gz',
#                      f'/home/abhijit/nas_drive/Abhijit/Jyotirmay/ukb_processed/data_/{v}.nii.gz')
    except Exception as e:
        print(e, ix, v)
        continue

df.to_csv('/home/abhijit/Jyotirmay/my_thesis/projects/MC_dropout_quicknat/reports/MC_dropout_quicknat_UKB_v2/UKB/0_0.0_modified_concat_report_final.csv')

Processing 19852/19853.

In [ ]:
len(f_paths)

In [ ]:
f_paths

In [ ]:
import torch

In [ ]:
model = torch.load('/home/abhijit/Jyotirmay/my_thesis/projects/full_bayesian/saved_models/full_bayesian_KORA_v2.pth.tar')

In [ ]:
torch.save(model.state_dict(), '/home/abhijit/Jyotirmay/whole_body_pipeline/nn_api/nn_api/pytorch_models/full_bayesian.model')